In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/26 21:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/26 21:20:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
!gzip -dc fhvhv_tripdata_2021-01.csv.gz > fhvhv_tripdata_2021-01.csv

In [ ]:
!wc -l fhvhv_tripdata_2021-01.csv

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.schema
df.head()

In [ ]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [4]:
import pandas as pd

In [ ]:
df_pandas = pd.read_csv('head.csv.gz')

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema

Integer - 4 bytes
Long - 8 bytes

In [5]:
from pyspark.sql import types

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.head(10)

In [ ]:
df = df.repartition(24)

In [6]:
df.write.parquet('fhvhv/2021/01/')

NameError: name 'df' is not defined

In [6]:
df = spark.read.parquet('fhvhv/2021/01/')

In [7]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [8]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
.filter(df.hvfhs_license_num=='HV0003')\
.show()

# select * from df where hvfhs_license_num = HV0003

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 10:20:02|2021-01-03 10:25:03|         158|         249|
|2021-01-01 16:37:10|2021-01-01 16:44:05|         232|          45|
|2021-01-01 03:21:16|2021-01-01 03:25:26|         255|         255|
|2021-01-01 03:55:46|2021-01-01 04:06:04|          83|         138|
|2021-01-02 13:19:32|2021-01-02 13:56:27|         191|         265|
|2021-01-03 05:32:25|2021-01-03 06:00:44|         225|          29|
|2021-01-02 08:31:42|2021-01-02 08:39:07|          51|          51|
|2021-01-01 23:10:05|2021-01-01 23:15:49|         231|         261|
|2021-01-01 18:51:10|2021-01-01 19:11:15|          42|          32|
|2021-01-01 02:33:04|2021-01-01 03:06:58|         127|         174|
|2021-01-02 21:38:58|2021-01-02 21:49:25|         244|         262|
|2021-01-04 07:35:24|2021-01-04 08:02:11|       

SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [9]:
from pyspark.sql import functions as F

In [10]:
df.withColumn('pickup_date',F.to_date(df.pickup_datetime)) \
.withColumn('dopoff_date', F.to_date(df.dropoff_datetime)) \
.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-----------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dopoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-----------+
|           HV0005|              B02510|2021-01-02 06:28:08|2021-01-02 06:43:35|         218|         135|   NULL| 2021-01-02| 2021-01-02|
|           HV0005|              B02510|2021-01-03 14:30:56|2021-01-03 14:44:15|          51|         265|   NULL| 2021-01-03| 2021-01-03|
|           HV0005|              B02510|2021-01-02 14:16:09|2021-01-02 14:25:21|         165|          89|   NULL| 2021-01-02| 2021-01-02|
|           HV0003|              B02884|2021-01-03 10:20:02|2021-01-03 10:25:03|         158|         249|   NULL| 2021-01-03| 2021-01-03|
|           HV0005|        

In [11]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 06:28:08|2021-01-02 06:43:35|         218|         135|   NULL|
|           HV0005|              B02510|2021-01-03 14:30:56|2021-01-03 14:44:15|          51|         265|   NULL|
|           HV0005|              B02510|2021-01-02 14:16:09|2021-01-02 14:25:21|         165|          89|   NULL|
|           HV0003|              B02884|2021-01-03 10:20:02|2021-01-03 10:25:03|         158|         249|   NULL|
|           HV0005|              B02510|2021-01-03 17:10:31|2021-01-03 17:23:13|          63|          77|   NULL|
|           HV0003|              B02872|2021-01-01 16:37:10|2021-01-01 16:44:05|

In [12]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [13]:
crazy_stuff('B02884')

's/b44'

In [14]:
#User Defined functions
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [15]:
#showing we can create new columns using the Funcitons library from pyspark.sql

df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-02|  2021-01-02|         218|         135|
|  e/9ce| 2021-01-03|  2021-01-03|          51|         265|
|  e/9ce| 2021-01-02|  2021-01-02|         165|          89|
|  s/b44| 2021-01-03|  2021-01-03|         158|         249|
|  e/9ce| 2021-01-03|  2021-01-03|          63|          77|
|  e/b38| 2021-01-01|  2021-01-01|         232|          45|
|  e/b3e| 2021-01-01|  2021-01-01|         255|         255|
|  e/9ce| 2021-01-01|  2021-01-01|         236|          41|
|  s/b3d| 2021-01-01|  2021-01-01|          83|         138|
|  s/acd| 2021-01-02|  2021-01-02|         191|         265|
|  e/b42| 2021-01-03|  2021-01-03|         225|          29|
|  e/9ce| 2021-01-01|  2021-01-01|         254|         265|
|  s/acd| 2021-01-02|  2021-01-02|          51|          51|
|  s/acd| 2021-01-01|  2

In [16]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [17]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,
